## Rock Paper Scissor Game Analyzer

# Overview

| Strategy Name                            | Explanation                                                                  | Type                       |
| ---------------------------------------- | ---------------------------------------------------------------------------- | -------------------------- |
| **s_random**                             | Chooses R, P, or S uniformly at random.                                      | Random                     |
| **s_constant_R**                         | Always chooses Rock.                                                         | Static              |
| **s_constant_P**                         | Always chooses Paper.                                                        | Static              |
| **s_constant_S**                         | Always chooses Scissors.                                                     | Static              |
| **s_constant(choice)**                   | Always chooses the move(s) specified in `choice`.                            | Static              |
| **s_probabilities(probabilities)**       | Chooses R, P, or S according to given probabilities `[Pr, Pp, Ps]`.          | Random/Biased              |
| **s_uniform_between(choices)**           | Chooses randomly between the given subset of moves.                          | Random                     |
| **s_special_choice(special_choice, p)**  | Chooses `special_choice` with probability `p`, others with `(1-p)/2`.        | Biased                     |
| **s_main_choice(main_choice)**           | Chooses `main_choice` with high probability (0.8).                           | Biased                     |
| **s_neglected_choice(neglected_choice)** | Chooses `neglected_choice` with low probability (0.2).                       | Biased                     |
| **s_mimic_opponent**                     | Chooses whatever the opponent played in the previous round.                  | Reactive                   |
| **s_last_move_wp_p(context, p)**         | Chooses the player’s last move with probability `p`, otherwise switches.     | Reactive                   |
| **s_tend_to_repeat**                     | Favors repeating the player’s last move.                                     | Reactive                   |
| **s_tend_to_change**                     | Favors switching from the player’s last move.                                | Reactive                   |
| **s_never_repeat**                       | Never chooses the player’s last move; picks from the other two.              | Reactive                   |
| **s_result_dependent**                   | Repeats last move if it won the previous round, otherwise changes.           | Reactive                   |
| **s_human_like**                         | Biased probabilities: R>P>S, adjusts slightly based on last move and result. | Hybrid (Biased + Reactive) |


# Imports

In [2]:
import random as rd
import numpy as np

# Global Variables

In [3]:
RPS = ['R', 'P', 'S'] # R = Rock, P = Paper, S = scissor
#last_move = [rd.choice(list(RPS)), rd.choice(list(RPS))]
#last_winner = 0

# Important Functions

In [4]:
# This function prints an error message and terminates the program
def print_error_and_exit(message = "something wrong happened"):
    print(message)
    exit()


In [5]:

# This function checks if it's parameter is a subset of RPS
# If yes it returns True, 
# Otherwise it prints that the parameter passed is an invalid choice and exits from the program

def check_if_valid_choice(to_check):
    set_to_check = set(to_check)
    if set_to_check.issubset(RPS):
        return True
    print_error_and_exit("invalid choice passed")

In [6]:

# This function checks if it's parameter is a valid probability
# If yes it returns True, 
# Otherwise it prints that the parameter passed is an invalid choice and exits from the program

def check_if_valid_probability(to_check):
    if isinstance(to_check, (int, float)):
        to_check = [to_check]
    for p in to_check:
        if not (0 <= p <= 1):
            print_error_and_exit("invalid probability passed")
    return True

In [7]:
# Function to decide winner between two moves
def get_winner(move1, move2):
    if move1 == move2:
        return 0
    elif (move1 == 'R' and move2 == 'S') or \
         (move1 == 'S' and move2 == 'P') or \
         (move1 == 'P' and move2 == 'R'):
        return 1
    else:
        return 2

In [8]:
def normalize(probs):
    total = sum(probs)
    if total == 0:
        print_error_and_exit("sum of probabilities cannot be zero")
    return [p / total for p in probs]

# Strategies

In [9]:
# Choose R with probability P0
# Choose P with probability P1
# Choose S with probability P2

def s_probabilities(probabilities):
    if len(probabilities) != 3:
        print_error_and_exit("wrong number of probabilities")
    check_if_valid_probability(probabilities)
    probabilities = normalize(probabilities)
    return rd.choices(RPS, weights=probabilities, k=1)[0]

In [10]:
# Choose randomly between the subset of choices passed
def s_uniform_between(choices):
    check_if_valid_choice(choices)
    return rd.choice(list(choices))


In [11]:
# Random Strategy: 
# Choose each of the three options (R,P,S) with probability 1/3
def s_random(context=None):
    return s_uniform_between(RPS)

In [12]:
# Constant Strategy:
# Choose the same thing all the time (passed as a parameter)
# And I made other 3 functions - one for each choice

def s_constant(choice):
    return s_uniform_between(choice)

def s_constant_R(context=None): return 'R'
def s_constant_P(context=None): return 'P'
def s_constant_S(context=None): return 'S'

In [13]:
#special choice strategy: 
#choose 1 thing with probability p,
#choose the others with probability (1-p)/2
#derived strategies: main_choice (high p), neglected_choice (low p)

def s_special_choice(special_choice, p):
    check_if_valid_probability(p)
    check_if_valid_choice(special_choice)
    probs = [0, 0, 0]
    idx = RPS.index(special_choice)
    probs[idx] = p
    others = (1 - p) / 2
    for i in range(3):
        if i != idx:
            probs[i] = others
    return s_probabilities(probs)

def s_main_choice(main_choice):
    return s_special_choice(main_choice, 0.8)

def s_neglected_choice(neglected_choice):
    return s_special_choice(neglected_choice, 0.2)

In [14]:
# Choose what the opponent chose last move
def s_mimic_opponent(context):
    return context["last_move_opponent"]

In [15]:
# Choose like you did previous round with probability p
# Three strategies derived from it are:
# 1. change your choice with high probability (tend to not repeat)
# 2. change your choice with low probability (tend to stick to last choice)
# 3. never repeat: never choose what you chose last round

def s_last_move_wp_p(context, p):
    return s_special_choice(context["last_move_self"], p)

def s_tend_to_change(context):
    return s_neglected_choice(context["last_move_self"])

def s_tend_to_repeat(context):
    return s_main_choice(context["last_move_self"])

def s_never_repeat(context):
    return s_uniform_between(set(RPS).difference(context["last_move_self"]))


In [16]:
# tend to repeat last move if it led to winning the round
# otherwise tend to choose something else

def s_result_dependent(context):
    if context["last_winner"] == context["player_n"]:
        return s_tend_to_repeat(context)
    return s_tend_to_change(context)

In [17]:
# "Human like" strategy (what I think)
# choose R more often than P, and choose P more often than S
# tend to repeat last choice when it led to winning,
# but tend to change it if it led to losing
# anyway relatively low probability to repeat last move (even when winning)

def s_human_like(context):
    probabilities = np.array([0.5, 0.3, 0.2])  # R>P>S
    move_to_index = {'R': 0, 'P': 1, 'S': 2}
    last = context["last_move_self"]
    idx = move_to_index[last]
    probabilities[idx] /= 4
    if context["last_winner"] == 0:
        probabilities[idx] *= 2
    probabilities = probabilities / probabilities.sum()
    return s_probabilities(probabilities)




# Play Function

In [18]:
def play(strategy1, strategy2, n=100):
    global last_move, last_winner
    last_move = [rd.choice(RPS), rd.choice(RPS)]
    last_winner = 0

    wins = [0, 0]
    ties = 0

    for _ in range(n):
        ctx1 = {
            "last_move_self": last_move[0],
            "last_move_opponent": last_move[1],
            "last_winner": last_winner,
            "player_n": 1
        }
        ctx2 = {
            "last_move_self": last_move[1],
            "last_move_opponent": last_move[0],
            "last_winner": last_winner,
            "player_n": 2
        }

        # Get moves
        try:
            move1 = strategy1(ctx1)
        except TypeError:
            move1 = strategy1()
        try:
            move2 = strategy2(ctx2)
        except TypeError:
            move2 = strategy2()

        if move1 not in RPS or move2 not in RPS:
            print_error_and_exit(f"Invalid move: {move1}, {move2}")

        # Decide winner
        winner = get_winner(move1, move2)
        if winner == 1:
            wins[0] += 1
            last_winner = 1
        elif winner == 2:
            wins[1] += 1
            last_winner = 2
        else:
            ties += 1
            last_winner = 0

        last_move = [move1, move2]

    print("\n--- Game Results ---")
    print(f"Player 1 wins: {wins[0]} ({wins[0]/n*100:.1f}%)")
    print(f"Player 2 wins: {wins[1]} ({wins[1]/n*100:.1f}%)")
    print(f"Ties: {ties} ({ties/n*100:.1f}%)")

    return wins, ties


# Example Runs

In [19]:
# Example Runs
play(s_human_like, s_random, n=10000)
play(s_tend_to_repeat, s_tend_to_change, n=10000)
play(s_mimic_opponent, s_random, n=10000)


--- Game Results ---
Player 1 wins: 3283 (32.8%)
Player 2 wins: 3389 (33.9%)
Ties: 3328 (33.3%)

--- Game Results ---
Player 1 wins: 3369 (33.7%)
Player 2 wins: 3323 (33.2%)
Ties: 3308 (33.1%)

--- Game Results ---
Player 1 wins: 3402 (34.0%)
Player 2 wins: 3287 (32.9%)
Ties: 3311 (33.1%)


([3402, 3287], 3311)

# Human like strategy against other strategies:

In [25]:
# human like vs everything else:
print("vs random")
play(s_human_like, s_random, n=10000)
print()

print("vs repeat variations")
play(s_human_like, s_never_repeat, n=10000)
play(s_human_like, s_tend_to_repeat, n=10000)
play(s_human_like, s_tend_to_change, n=10000)
print()

print("vs mimic opponent")
play(s_human_like, s_mimic_opponent, n = 10000)
print()

print("vs human like")
play(s_human_like, s_human_like, n = 10000)
print()

print("vs constants")
play(s_human_like, s_constant_R, n = 10000)
play(s_human_like, s_constant_P, n = 10000)
play(s_human_like, s_constant_S, n = 10000)
print()

print("vs result dependent")
play(s_human_like, s_result_dependent, n = 10000)


vs random

--- Game Results ---
Player 1 wins: 3423 (34.2%)
Player 2 wins: 3304 (33.0%)
Ties: 3273 (32.7%)

vs repeat variations

--- Game Results ---
Player 1 wins: 3359 (33.6%)
Player 2 wins: 3469 (34.7%)
Ties: 3172 (31.7%)

--- Game Results ---
Player 1 wins: 3202 (32.0%)
Player 2 wins: 3291 (32.9%)
Ties: 3507 (35.1%)

--- Game Results ---
Player 1 wins: 3378 (33.8%)
Player 2 wins: 3358 (33.6%)
Ties: 3264 (32.6%)

vs mimic opponent

--- Game Results ---
Player 1 wins: 4358 (43.6%)
Player 2 wins: 4141 (41.4%)
Ties: 1501 (15.0%)

vs human like

--- Game Results ---
Player 1 wins: 3403 (34.0%)
Player 2 wins: 3344 (33.4%)
Ties: 3253 (32.5%)

vs constants

--- Game Results ---
Player 1 wins: 2937 (29.4%)
Player 2 wins: 2216 (22.2%)
Ties: 4847 (48.5%)

--- Game Results ---
Player 1 wins: 2284 (22.8%)
Player 2 wins: 4218 (42.2%)
Ties: 3498 (35.0%)

--- Game Results ---
Player 1 wins: 4316 (43.2%)
Player 2 wins: 3237 (32.4%)
Ties: 2447 (24.5%)

vs result dependent

--- Game Results ---
Play

([3641, 2833], 3526)

Analyze Results of the section:
- Against random strategies, it shows no consistent advantage, with wins, losses, and ties all close to one-third.
- When facing reactive strategies, such as tend_to_repeat, tend_to_change, or never_repeat, outcomes remain balanced, though tie percentages fluctuate slightly.
- Notably, when playing against s_mimic_opponent, ties drop dramatically to around 15%, even though the win/loss ratio remains roughly even, highlighting that human_like’s probabilistic bias desynchronizes mimic responses.
- Against static constant strategies, human_like exploits its internal bias, winning decisively against constants it favors (R or S) and losing against those that counter its bias (P)
- Finally, human_like has a moderate advantage against result-dependent strategies, suggesting that combining a biased choice distribution with minor adaptive behavior can outperform strategies that rely solely on previous outcomes.